In [3]:
import numpy as np
import pandas as pd
from matplotlib.patches import Patch
import itertools
import pickle
from matplotlib import pyplot as plt
import datetime
from scipy import optimize
import sys
sys.path.append('../SEIR_full/')
sys.path.append('..')
import SEIR_full as mdl
import SEIR_full.model_class as mdl
import datetime as dt

In [4]:
plt.style.use('dean_style.conf')

In [115]:
IL_pop = 9136000

# Imports model's predictions

In [6]:
with open('../Data/results/23_4_20_12_36.pickle','rb') as pickle_file:
    inter_data = pickle.load(pickle_file)

In [16]:
inter_data.keys()

dict_keys([('50_no_60+risk_no_school_nokid_inter', 'ub'), ('50_no_60+risk_no_school_kid06_inter', 'ub'), ('100_no_60+risk_inter', 'ub'), ('50_no_60+risk_no_school_nokid_inter', 'base'), ('50_no_60+risk_no_school_kid06_inter', 'base'), ('100_no_60+risk_inter', 'base'), ('50_no_60+risk_no_school_nokid_inter', 'lb'), ('50_no_60+risk_no_school_kid06_inter', 'lb'), ('100_no_60+risk_inter', 'lb')])

In [78]:
region_age_ga_dict = mdl.get_opposite_dict(mdl.GA,list(itertools.product(mdl.G.values(),mdl.A.values())))
region_index = {v:k for k,v in mdl.G.items()}
init_region_pop = mdl.shrink_array_sum(mapping_dic=mdl.region_dict,array_to_shrink=mdl.population_size)
init_region_age_pop = mdl.shrink_array_sum(mapping_dic=mdl.region_age_dict,array_to_shrink=mdl.population_size)

In [114]:
init_region_age_pop.sum()

1.0

# Map dashboard

In [171]:
def map_table(res_mdl, inter_code, col_name='mean', start_day='2020-02-20'):
    
    tot_I = res_mdl['Is'] + res_mdl['Ia'] + res_mdl['Ie']
    date_range = pd.date_range(start_day, periods = tot_I.shape[0], freq='d')
    
    # Creating table for map, by region:
    table = pd.DataFrame(index = pd.MultiIndex.from_tuples(itertools.product(date_range, mdl.G.values(),list(mdl.A.values()) + ['total']),
                                                           names=['date','region','age']), 
                                     columns=[col_name])
    # filling df, cases per 1K
    for index in list(table.index):
#         print(index)
#         print(type(tot_I[int(np.where(date_range == index[0])[0]),mdl.region_dict[index[1]]].sum()))
#         print([int(np.where(date_range == index[0])[0]),mdl.region_dict[index[1]]])
        
        # if total area value needed
        if index[2] == 'total':
            table.loc[index] = (tot_I[int(np.where(date_range == index[0])[0]),mdl.region_dict[index[1]]].sum()
                                / init_region_pop[region_index[index[1]]]) * 1000
        else:
            table.loc[index] = (tot_I[int(np.where(date_range == index[0])[0]),mdl.region_age_dict[index[1],index[2]]].sum()
                                / init_region_age_pop[region_age_ga_dict[index[1], index[2]]]) * 1000
    
    # calculate weekly cases
    # 'daysoffset' will container the weekday, as integers
    table['daysoffset'] = table.apply(lambda x: x.name[0].weekday(),axis=1)
    # We apply, row by row (axis=1) a timedelta operation
    table['week_start'] = table.apply(lambda x: x.name[0] - dt.timedelta(days=x['daysoffset']), axis=1)
    
    table.reset_index(inplace=True)
    table_groupped = table.groupby(['week_start','region','age'])[col_name]
    table_groupped = table_groupped.apply(lambda x: (x.max() + x.min())/2)
    table_groupped =  table_groupped.to_frame()
    table_groupped['scenario_idx'] = inter_code
    
    return(table_groupped[pd.to_datetime('today'):])

In [188]:
# scenario_name = '50_no_60+risk_no_school_nokid_inter'
# scenario_name = '50_no_60+risk_no_school_kid06_inter'
scenario_name = '100_no_60+risk_inter'
model_sets = ['base']

In [184]:
inter_dict_map = {}

In [189]:
for s in model_sets:
    inter_dict_map[scenario_name] = map_table(inter_data[scenario_name,s], 
                                     scenario_name, col_name=s, start_day='2020-02-20').reset_index()
    

In [190]:
inter_df_list = [v for v in inter_dict_map.values()]
inter_df_concat = pd.concat(inter_df_list)

In [192]:
# Save
inter_df_concat.to_csv('../Data/results/map_data_upload_230420.csv')

# Age distribution

In [118]:
def age_distribution_table(res_mdl, inter_code, col_name='mean', start_day='2020-02-20'):
    
    tot_I = res_mdl['Is'] + res_mdl['Ia'] + res_mdl['Ie']
    date_range = pd.date_range(start_day, periods = tot_I.shape[0], freq='d')
    
    # Creating table for map, by region:
    table = pd.DataFrame(index = pd.MultiIndex.from_tuples(itertools.product(date_range,list(mdl.A.values())),
                                                           names=['date','age']), 
                                     columns=[col_name])

    for index in list(table.index):
#         print(index)
#         print(type(tot_I[int(np.where(date_range == index[0])[0]),mdl.region_dict[index[1]]].sum()))
#         print([int(np.where(date_range == index[0])[0]),mdl.region_dict[index[1]]])
        
        table.loc[index] = (tot_I[int(np.where(date_range == index[0])[0]),mdl.age_dict[index[1]]].sum())*IL_pop
    
    # calculate weekly cases
    # 'daysoffset' will container the weekday, as integers
    table['daysoffset'] = table.apply(lambda x: x.name[0].weekday(),axis=1)
    # We apply, row by row (axis=1) a timedelta operation
    table['week_start'] = table.apply(lambda x: x.name[0] - dt.timedelta(days=x['daysoffset']), axis=1)
    
    table.reset_index(inplace=True)
    table_groupped = table.groupby(['week_start','age'])[col_name]
    table_groupped = table_groupped.apply(lambda x: np.ceil((x.max() + x.min())/2))
    table_groupped =  table_groupped.to_frame()
    table_groupped['scenario_idx'] = inter_code
    
    return(table_groupped[pd.to_datetime('today'):])

In [244]:
def combine_ub_lb_ml(ub_df, lb_df, ml_df, col_list=['ub','lb','ml']):
    
    combined_df = ml_df.copy()
    combined_df = combined_df.merge(ub_df, left_index=True, right_index=True)
    combined_df = combined_df.merge(lb_df, left_index=True, right_index=True)
#     print(combined_df)
    # fixing LB/Mean/UB
    combined_df['new_lb'] = combined_df[col_list].apply(lambda x: x.min(), axis=1)
    combined_df['new_ub'] = combined_df[col_list].apply(lambda x: x.max(), axis=1)
    combined_df['new_ml'] = combined_df[col_list].apply(lambda x: sorted([x[0],x[1],x[2]])[1], axis=1)
    
    return combined_df[['new_lb','new_ub','new_ml','scenario_idx']]
    

In [250]:
# scenario_name = '50_no_60+risk_no_school_nokid_inter'
# scenario_name = '50_no_60+risk_no_school_kid06_inter'
scenario_name = '100_no_60+risk_inter'
model_sets = ['base','ub','lb']

In [246]:
inter_dict_age_dis = {}

In [251]:
curr_inter_dict = {}
for s in model_sets:
    curr_inter_dict[s] = age_distribution_table(inter_data[scenario_name,s], 
                                     scenario_name, col_name=s, start_day='2020-02-20')

    
# combining ub,lb,ml to one DF:

inter_dict_age_dis[scenario_name] = combine_ub_lb_ml(curr_inter_dict['ub'],curr_inter_dict['lb'],curr_inter_dict['base'],
                                                col_list=model_sets).reset_index()

In [252]:
inter_df_list_age_dis = [v for v in inter_dict_age_dis.values()]
inter_df_age_dis = pd.concat(inter_df_list_age_dis)

In [253]:
inter_df_age_dis

,week_start,age,new_lb,new_ub,new_ml,scenario_idx
0,2020-04-27,0-4,121.0,798.0,283.0,50_no_60+risk_no_school_nokid_inter
1,2020-04-27,10-19,396.0,2717.0,1054.0,50_no_60+risk_no_school_nokid_inter
2,2020-04-27,20-29,170.0,1467.0,509.0,50_no_60+risk_no_school_nokid_inter
3,2020-04-27,30-39,155.0,1279.0,457.0,50_no_60+risk_no_school_nokid_inter
4,2020-04-27,40-49,78.0,1212.0,228.0,50_no_60+risk_no_school_nokid_inter
...,...,...,...,...,...,...
103,2020-07-13,40-49,18855.0,122354.0,29959.0,100_no_60+risk_inter
104,2020-07-13,5-9,33718.0,89557.0,41667.0,100_no_60+risk_inter
105,2020-07-13,50-59,13036.0,73995.0,21117.0,100_no_60+risk_inter
106,2020-07-13,60-69,17333.0,81036.0,25722.0,100_no_60+risk_inter


In [254]:
# Save
inter_df_age_dis.to_csv('../Data/results/age_data_upload_230420.csv')

# Ventilators graph

In [255]:
def vents_graph(res_mdl, inter_code, tracking='Vents' ,col_name='mean', curr_vents=None, start_day='2020-02-20', 
                inter_start_date='2020-04-20', fix_vents=False):
    
    tot_vents = np.ceil(res_mdl[tracking].sum(axis=1) * IL_pop) 
    date_range = pd.date_range(start_day, periods = tot_vents.shape[0], freq='d')
    # fixing patients no.
    if fix_vents:
        model_vents_proj = tot_vents[int(np.where(date_range == inter_start_date)[0])]
        tot_vents += max(curr_vents - model_vents_proj, 0)

    
    # creates DF with results
    vents_dict = {}
    vents_dict['date'] = date_range
    vents_dict[col_name] = tot_vents
    vents_df = pd.DataFrame.from_dict(vents_dict)
    
    # 'daysoffset' will container the weekday, as integers
    vents_df['daysoffset'] = vents_df['date'].apply(lambda x: x.weekday())
    # We apply, row by row (axis=1) a timedelta operation
    vents_df['week_start'] = vents_df.apply(lambda x: x['date'] - dt.timedelta(days=x['daysoffset']), axis=1)
    
    vents_df_grop = vents_df.groupby('week_start')[col_name]
    vents_df_grop = vents_df_grop.max().to_frame()
    vents_df_grop['scenario_idx'] = inter_code
    return (vents_df_grop[pd.to_datetime('today'):])

In [261]:
# scenario_name = '50_no_60+risk_no_school_nokid_inter'
# scenario_name = '50_no_60+risk_no_school_kid06_inter'
scenario_name = '100_no_60+risk_inter'
model_sets = ['base','ub','lb']

In [257]:
inter_dict_vents_dict = {}

In [262]:
curr_inter_dict = {}
for s in model_sets:
    curr_inter_dict[s] = vents_graph(inter_data[scenario_name,s], 
                                     scenario_name, col_name=s, curr_vents=113, start_day='2020-02-20', fix_vents=True)

    
# combining ub,lb,ml to one DF:

inter_dict_vents_dict[scenario_name] = combine_ub_lb_ml(curr_inter_dict['ub'],curr_inter_dict['lb'],curr_inter_dict['base'],
                                                col_list=model_sets).reset_index()

In [263]:
inter_df_list_vents_dis = [v for v in inter_dict_vents_dict.values()]
inter_df_vents_dis = pd.concat(inter_df_list_vents_dis)

In [264]:
inter_df_vents_dis

,week_start,new_lb,new_ub,new_ml,scenario_idx
0,2020-04-27,104.0,175.0,110.0,50_no_60+risk_no_school_nokid_inter
1,2020-05-04,96.0,165.0,107.0,50_no_60+risk_no_school_nokid_inter
2,2020-05-11,90.0,160.0,105.0,50_no_60+risk_no_school_nokid_inter
3,2020-05-18,85.0,160.0,103.0,50_no_60+risk_no_school_nokid_inter
4,2020-05-25,82.0,164.0,102.0,50_no_60+risk_no_school_nokid_inter
5,2020-06-01,79.0,169.0,102.0,50_no_60+risk_no_school_nokid_inter
6,2020-06-08,77.0,174.0,102.0,50_no_60+risk_no_school_nokid_inter
7,2020-06-15,76.0,180.0,102.0,50_no_60+risk_no_school_nokid_inter
8,2020-06-22,74.0,187.0,102.0,50_no_60+risk_no_school_nokid_inter
9,2020-06-29,73.0,194.0,102.0,50_no_60+risk_no_school_nokid_inter


In [81]:
# Save
inter_df_vents_dis.to_csv('../Data/results/vents_data_upload_230420.csv')

# Hospitalization graph

In [87]:
# scenario_name = '50_no_60+risk_no_school_nokid_inter'
# scenario_name = '50_no_60+risk_no_school_kid06_inter'
scenario_name = '100_no_60+risk_inter'
model_sets = ['base','ub','lb']

In [83]:
inter_dict_vents_dict = {}

In [88]:
curr_inter_dict = {}
for s in model_sets:
    curr_inter_dict[s] = vents_graph(inter_data[scenario_name,s], 
                                     scenario_name, tracking='H', col_name=s, curr_vents=113, start_day='2020-02-20', fix_vents=True)

    
# combining ub,lb,ml to one DF:

inter_dict_vents_dict[scenario_name] = combine_ub_lb_ml(curr_inter_dict['ub'],curr_inter_dict['lb'],curr_inter_dict['base'],
                                                col_list=model_sets).reset_index()

In [89]:
inter_df_list_vents_dis = [v for v in inter_dict_vents_dict.values()]
inter_df_vents_dis = pd.concat(inter_df_list_vents_dis)

In [90]:
inter_df_vents_dis

,week_start,new_lb,new_ub,new_ml,scenario_idx
0,2020-04-27,154.0,1271.0,468.0,50_no_60+risk_no_school_nokid_inter
1,2020-05-04,135.0,1210.0,410.0,50_no_60+risk_no_school_nokid_inter
2,2020-05-11,123.0,1199.0,370.0,50_no_60+risk_no_school_nokid_inter
3,2020-05-18,115.0,1223.0,341.0,50_no_60+risk_no_school_nokid_inter
4,2020-05-25,110.0,1257.0,319.0,50_no_60+risk_no_school_nokid_inter
5,2020-06-01,108.0,1299.0,302.0,50_no_60+risk_no_school_nokid_inter
6,2020-06-08,107.0,1344.0,287.0,50_no_60+risk_no_school_nokid_inter
7,2020-06-15,108.0,1392.0,275.0,50_no_60+risk_no_school_nokid_inter
8,2020-06-22,110.0,1443.0,264.0,50_no_60+risk_no_school_nokid_inter
9,2020-06-29,113.0,1495.0,255.0,50_no_60+risk_no_school_nokid_inter


In [265]:
# Save
inter_df_vents_dis.to_csv('../Data/results/hospitalization_data_upload_230420.csv')

# County id -> name mapper

In [268]:
region_name = {'11':'Jerusalem and sub.', '11_betshemesh':'Bet Shemesh', '21':'Zefat', '22':'Kinneret',
               '23':'Jezreel Valley', '24':'Akko', '29':'Ramat Hagolan', '31':'Haifa', '32':'Hadera',
               '41': 'Sharon', '42':'Petah Tiqwa', '43':'Ramla', '44':'Rehovot', '51_tlv':'Tel Aviv - Yafo',
               '51_bb': 'Bnei Brak', '51':'TLV suburbs', '61':'Ashqelon', '62':'Beer Sheva Jewish',
               '62_arab':'Beer Sheva Arabs','71': 'Judea and Samaria'}


In [269]:
region_name_df = pd.DataFrame.from_dict(region_name,orient='index')

In [271]:
region_name_df.to_csv('../Data/dashboard/region_name_mapper_county.csv')